<a href="https://colab.research.google.com/github/jyp1111/TIL/blob/master/nlp/document_summerize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from itertools import permutations
import numpy as np

In [ ]:
class textrank_sen_1:    
    def __init__(self,document):
        self.document=document
    def sen_ls(self):
        L_sen=self.document.split(". ")
        L_sen[-1]=L_sen[-1].strip(".")
        L_sen_token_ls=[x.split() for x in L_sen]

        return L_sen_token_ls

    def connected_matrix(self):
        m=len(self.sen_ls())
        Mat=np.zeros((m,m))

        for v in list(permutations(range(m),2)):
            Mat[v[0]][v[1]]=len(set(self.sen_ls()[v[0]]).intersection(set(self.sen_ls()[v[1]])))/len(set(self.sen_ls()[v[0]]).union(set(self.sen_ls()[v[1]])))
            
        result_Mat=np.array([Mat[i]/sum(Mat[i]) for i,_ in enumerate(Mat)])

        return result_Mat

    def score(self,d,num_epoch,threshold):
        m=len(self.sen_ls())
        # score=np.ones(m)*10
        score=np.random.randint(1,100,size=m)
        table=[]
        for i in range(num_epoch):
            new_score=(1-d)+np.dot(self.connected_matrix().T,d*score)
            if (np.abs(score-new_score)).mean()<threshold:
                break
            score=new_score
        for i,x in enumerate(self.sen_ls()):
            sen=' '.join(x)
            table.append({"sentence":sen,"score":new_score[i]})
        table=pd.DataFrame(table)
        return table
    
    def ideal_score(self,d): # inverse 계산이 원래는 힘들기 때문에 이론상 함수
        m=len(self.sen_ls())
        ideal_score=np.dot(np.linalg.inv(np.identity(m)-self.connected_matrix().T*d)*(1-d),np.ones(m))

        table=[]
        for i,x in enumerate(self.sen_ls()):
            sen=' '.join(x)
            table.append({"sentence":sen,"score":ideal_score[i]})
        table=pd.DataFrame(table)

        return table


In [ ]:
document="딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."

In [ ]:
textrank_sen_1(document).score(d=0.85,num_epoch=1000,threshold=0.0001)

,sentence,score
0,딸기 바나나 사과 파인애플 수박,1.411038
1,바나나 사과 딸기 포도,1.025095
2,복숭아 수박,0.286306
3,파인애플 사과 딸기 바나나,1.279582


In [ ]:
textrank_sen_1(document).ideal_score(d=0.85)

,sentence,score
0,딸기 바나나 사과 파인애플 수박,1.410321
1,바나나 사과 딸기 포도,1.024557
2,복숭아 수박,0.286224
3,파인애플 사과 딸기 바나나,1.278898
